In [1]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import pandas_datareader as pdr
from scipy.optimize import minimize
import json

In [2]:
argv = ["function_name", "MSFT AAPL GOOG AMZN XOM HSBC BRK-B JPM BAC WFC ", "200", "100000"]

In [3]:
%%time
'''
Based on : https://stackoverflow.com/questions/23450534/how-to-call-a-python-function-from-node-js
argv[1] : String of valid stock tickers in the portfolio
argv[2] : Duration window (in days) for stock variance calculations
argv[3] : Portfolio Capital (in cents)
'''
stocks = argv[1].split()
window = int(argv[2])
initial_capital = int(argv[3])

# Get data
end_date = datetime.today()
start_date = end_date - timedelta(window+2)
# stocks = ['MSFT','AAPL','GOOG','AMZN','XOM','HSBC','BRK-B','JPM','BAC','WFC']
stock_data = pdr.get_data_yahoo(symbols=stocks, start=start_date, end=end_date)
stock_data = stock_data.dropna()

# Calculate returns
for ticker in stock_data.columns.levels[1]:
    stock_data['Return', ticker] = stock_data['Adj Close', ticker].pct_change(1)
    
stock_data = stock_data.dropna()

tickers = []
invVar = []

# Calculate Inverse Variance
for ticker in stock_data.columns.levels[1]:
    tickers.append(ticker)
    invVar.append(stock_data['Return', ticker].var())

weights = invVar/sum(invVar)

portfolio_weights = dict()

for index, ticker in enumerate(tickers):
    portfolio_weights[ticker] = weights[index]
    
print(json.dumps(portfolio_weights))

{"AAPL": 0.14416191899165584, "AMZN": 0.09790331334192418, "BAC": 0.128711446215379, "BRK-B": 0.04599170279928909, "GOOG": 0.16114817200631926, "HSBC": 0.056094165578631415, "JPM": 0.09298832371857388, "MSFT": 0.09669161005264852, "WFC": 0.10270262647663952, "XOM": 0.07360672081893938}
Wall time: 5.83 s


In [7]:
%%time
'''
Based on : https://stackoverflow.com/questions/23450534/how-to-call-a-python-function-from-node-js
argv[1] : String of valid stock tickers in the portfolio
argv[2] : Duration window (in days) for stock variance calculations
argv[3] : Portfolio Capital (in cents)
'''
stocks = argv[1].split()
window = int(argv[2])
initial_capital = int(argv[3])

# Get data
end_date = datetime.today()
start_date = end_date - timedelta(window+2)
# stocks = ['MSFT','AAPL','GOOG','AMZN','XOM','HSBC','BRK-B','JPM','BAC','WFC']
stock_data = pdr.get_data_yahoo(symbols=stocks, start=start_date, end=end_date)
stock_data = stock_data.dropna()

# Calculate returns
for ticker in stock_data.columns.levels[1]:
    stock_data['Return', ticker] = stock_data['Adj Close', ticker].pct_change(1)
    
stock_data = stock_data.dropna()
# print(stock_data)
cov = stock_data.Return.cov()
# print(covariance)
tickers = list(cov.columns.values)


# Optimization
def cost(w):
    global cov
    portfolio_variance = np.dot(np.dot(w.T, cov), w)
    return portfolio_variance
 
bnds = [(0.0001, 0.9999)]*len(cov)
cons = ({'type': 'eq', 'fun': lambda w: 1.0 - sum(w)})
w0 = np.random.randn(len(cov))/len(cov)

result = minimize(cost, w0, method='SLSQP', bounds=bnds, constraints=cons, tol=1e-6)
# print(result.message)

portfolio_weights = dict()

if result.success:
    weights = result.x
    for index, ticker in enumerate(tickers):
        portfolio_weights[ticker] = weights[index]

    
print(json.dumps(portfolio_weights))

{"AAPL": 0.037817546145716516, "AMZN": 0.1026458677092782, "BAC": 0.03780294074128574, "BRK-B": 0.32528783074871764, "GOOG": 0.03783401748016401, "HSBC": 0.16940361614031776, "JPM": 0.037820095155821735, "MSFT": 0.0378346746714858, "WFC": 0.037830481796318725, "XOM": 0.17572292941089399}
Wall time: 5.26 s
